# ST 生成器

- 总结 ST (Structured text) 语言的语法
- 自动化生成 ST 代码
- 使用生成的代码测试 openplc 中使用的 iec2c
- 通过 AFL 进行进一步模糊测试
- 样本分析与源码调试


In [100]:
from fuzzingbook.Grammars import EXPR_EBNF_GRAMMAR, srange, convert_ebnf_grammar, Grammar, Expansion, CHARACTERS_WITHOUT_QUOTE
from fuzzingbook.Grammars import is_valid_grammar, exp_string
from fuzzingbook.GrammarFuzzer import GrammarFuzzer, EvenFasterGrammarFuzzer
from fuzzingbook.GrammarCoverageFuzzer import GrammarCoverageFuzzer
from fuzzingbook.bookutils import print_file
import numpy as np
import string
import subprocess
import os
import tempfile


## ST 语法

> 结构式文件编程语言（英语：Structured text）也称为ST语言，是为可编程逻辑控制器（PLC）设计的编程语言，是相关的IEC 61131-3标准中支援几种语言之一。结构式文件编程语言是支援块状结构（block structured）的高阶语言，以Pascal为基础，语法也类似Pascal。所有IEC 61131-3的语言都支援IEC61131通用元素（IEC61131 Common Elements）。其变数及函式呼叫是由IEC61131通用元素所定，因此同一个程式中可以使用IEC 61131-3中的不同语言。

使用 openplc Editor 软件编写并生成的一段 ST 代码如下，该代码可导入到 openplc Runtime 中运行。

```iecst
PROGRAM program0
  VAR
    PB1 : BOOL;
    PB2 : BOOL;
    LED : BOOL;
  END_VAR

  IF PB1 THEN
    LED := TRUE;
  END_IF;

  IF PB2 THEN
    LED := FALSE;
  END_IF;
END_PROGRAM


CONFIGURATION Config0

  RESOURCE Res0 ON PLC
    TASK task0(INTERVAL := T#20ms,PRIORITY := 0);
    PROGRAM instance0 WITH task0 : program0;
  END_RESOURCE
END_CONFIGURATION
```
PROGRAM 部分是我们主要考虑的。


### 数据类型

https://en.wikipedia.org/wiki/IEC_61131-3

#### 基本数据类型

- 整数 INT...
- 浮点数 REAL ...
- 时间 TIME ...
- 字符串 STRING ...
- 布尔 BOOL ...


#### 组合数据类型


- ARRAY
- STRUCT
- UNION
- Sub-range

```iecst
// 结构体
TYPE Rectangle :
    STRUCT
    TopLeft : Point;
    Height : INT;
    Width : INT;
    END_STRUCT;
END_TYPE

// 枚举
TYPE Color :
    (Red, White, Blue);
END_TYPE

// Sub-ranges
TYPE Angle :
    INT(-180..+180);
END_TYPE


// 数组
TYPE Display :
    ARRAY[1..768, 1..1024] OF Color;
END_TYPE

```


### 赋值语句 Assignments

```iecst
VarA := (VarB * 24 MOD 2 = 1) XOR (VarC <= VarD + 34);

(* AnArray is an array containing twenty integers *)
AnArray := 10(1), 5(2), 5(3);

VarA := AND(Var1, Var2, ..., VarN);
```
一些运算符可以有任何数量的参数（相同类型）


In [101]:
# A.3
ST_ASSIGNMENTS_EBNF: Grammar = {
    "<AssignmentStatement>":
        ["<Variable> := <Expression>\n"],
}

### 函数调用语句 Function calls


*函数调用语句并不是ST标准，也没有很严格的规范，这里先不考虑*

注意，ST中不允许递归调用函数。

> Note that, strictly speaking, the FUNCTION … END_FUNCTION and the VAR … END_VAR constructs mentioned in this paragraph do not belong to the Structured text programming language, as discussed earlier.

```iecst
FUNCTION_BLOCK FB_Timed_Counter
    VAR_INPUT
        // ...
    END_VAR
    
    VAR_OUTPUT
        // ...
    END_VAR
    
    VAR
        // ...
    END_VAR
        
    // Start of Function Block programming
    
END_FUNCTION_BLOCK

// 另一种写法？
FUNCTION Ave_REAL : REAL
    VAR_INPUT
        Input1, Input2 : REAL;
    END_VAR
    Ave_REAL := (Input1 + Input2) / 2;
END_FUNCTION
Average1 := Ave_REAL(5.0, 4.0);
Average2 := Ave_REAL(Input2 := 6.0);
(* Value 3.0 assigned to Average2 *)


```


In [102]:
# A.4
# NOTE: 子程序控制与函数调用，不考虑

### 条件分支语句 Conditional statements

注意 等于号 `=` 和 赋值号 `:=` 的区别


```iecst
IF VarB > 0 THEN
  IF VarC = 3 THEN
    VarA := TRUE;
  ELSE
    VarC := 3;
    VarA := FALSE;
  END_IF;
ELSIF VarB < 0 THEN
  VarA := FALSE;
ELSE
  VarC := 3;
  VarA := TRUE;
END_IF;


CASE Var233 OF
  0: ...;
  1: ...;
ELSE
  
END_CASE;

```

In [103]:
# A.5
_ST_IF_S = ["""
IF <Expression> THEN  
    <StatementList>
(ELSIF <Expression> THEN
    <StatementList>)*
(ELSE
    <StatementList>)?
END_IF"""]


_ST_CASE_S = ["""
CASE <Expression> OF
    <CaseElement>+
(ELSE
    <StatementList>)?
END_CASE"""]

ST_SELECTION_STATEMENT_EBNF: Grammar = {
    "<SelectionStatement>":
        ["<IfStatement>", "<CaseStatement>"],
    "<IfStatement>": 
        _ST_IF_S,
    "<CaseStatement>": 
        _ST_CASE_S,
    "<CaseElement>": 
        ["<CaseList> : <StatementList>\n"],
    "<CaseList>":
        ["<CaseListElement> (, <CaseListElement>)*"],
    "<CaseListElement>":
        # ["<Subrange>", "<SignedInteger>"], TODO
        ["<SignedInteger>"],
}



### 迭代循环语句 Iteration statements

3 种迭代循环语句

```iecst
FOR VarB := 1 TO VarC DO
  VarD := VarD + 1;
  VarE := 2 * VarE;
END_FOR;

FOR VarB := 100 TO 0 BY –2 DO
  VarD := VarD + VarB;
END_FOR;


WHILE VarA AND (VarD <= VarC) DO
  VarD := VarD + 1;
  VarE := 2 * VarE;
  VarA := VarE < 100;
END_WHILE;


REPEAT
  VarD := VarD + 1;
  VarE := 2 * VarE;
  VarA := VarE < 100;
UNTIL NOT(VarA AND (VarD <= VarC));
END_REPEAT;
```


In [104]:
# A.6

_ST_FOR_S = ["""
FOR <ControlVariable> := <ForList> DO
    <StatementList>
END_FOR"""]

_ST_WHILE_S = ["""
WHILE <Expression> DO
    <StatementList>
END_WHILE"""]

_ST_REPEAT_S = ["""
REPEAT
    <StatementList>
UNTIL <Expression>
END_REPEAT"""]

_ST_EXIT_S = ["EXIT"]

ST_ITERATION_STATEMENT_EBNF: Grammar = {
    "<IterationStatement>":
        ["<ForStatement>", 
         "<WhileStatement>", 
         "<RepeatStatement>",
         "<ExitStatement>",],
    "<ForStatement>":
        _ST_FOR_S,
    "<ControlVariable>":
        # ["<Identifier>"], TODO
        ["<Variable>"],
    "<ForList>":
        ["<Expression> TO <Expression> (BY <Expression>)?"],
    "<WhileStatement>":
        _ST_WHILE_S,
    "<RepeatStatement>":
        _ST_REPEAT_S,
    "<ExitStatement>":
        _ST_EXIT_S,
}



### 其他程序结构



#### 基础表达式


In [105]:
# 参照 A.1
ST_EXPRESSIONS_EBNF: Grammar = {
    "<Expression>":
        ["<XOR_Expression> (OR <XOR_Expression>)*"],
    "<XOR_Expression>":
        ["<AND_Expression> (XOR <AND_Expression>)*"],
    "<AND_Expression>":
        ["<Comparison> (AND <Comparison>)*"],
    "<Comparison>":
        ["<EquExpression> (= <EquExpression>)*"],
        #  "<EquExpression> (<> <EquExpression>)*",], TODO 转义
    "<EquExpression>":
        ["<AddExpression> (<ComparisonOperator> <AddExpression>)*"],
    "<ComparisonOperator>":
        ["<", ">", "<=", ">="],
    "<AddExpression>":
        ["<term> (<AddOperator> <term>)*"],
    "<AddOperator>":
        ["+", "-"],
    

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],
    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],
    "<factor>":
        ["<factor>", "(<expr>)", "<SignedInteger>(.<SignedInteger>)?", "<Variable>"],
    "<sign>":
        [" NOT ", " -"],
    "<SignedInteger>":
        ["<sign>?<digit>+"],
    "<digit>":
        srange(string.digits),
        
    # 类型变量
    "<Variable>":
        ["VarWillBeReplace"], 
        # ["<VariableINT>", "<VariableBOOL>", "<VariableREAL>"], 
    # "<VariableINT>":
    #     ["VarWillBeReplace"], 
    # "<VariableBOOL>":
    #     ["VarWillBeReplace"],
    # "<VariableREAL>":
    #     ["VarWillBeReplace"],
}




#### 整体结构

In [106]:
# 参照 A.2
ST_STATEMENTS_EBNF: Grammar = {
    "<StatementList>":
        ["(<Statement>;\n)+"],
    "<Statement>":
        ["<AssignmentStatement>\n", # 赋值语句
        #"<Function-statements>",  # 函数语句
         "<SelectionStatement>", # 条件语句
         "<IterationStatement>"], # 循环语句
}

## ST EBNF

### 根据 PASCAL EBNF 试试看

> 方括号划界可选构建体；`{}` 表示封闭构造的零或更多重复；`()` 表示构造的简单分组；`|` 表示从许多人中选择一个；定义中的文字使用粗体字体或双引号标记表示。

> EBNF operators – `?` becomes (0,1), `*` becomes (0,), and `+` becomes (1,). 


In [107]:
LOW_LEVEL_EBNF: Grammar = {
    "<variable-list>":
        ["<variable>", "<variable-list> , <variable>"],
    "<identifier-list>":
        ["<identifier>", "<identifier-list> , <identifier>"],
    "<expression-list>":
        ["<expression>", "<expression-list> , <expression>"],
    "<number>":
        ['<integer_number>', '<real_number>'],
    "<integer_number>":
        ['<digit_sequence>'],
    "<real_number>":
        ['<digit_sequence>(.<digit_sequence>)?<scale_factor>?'],
    "<scale_factor>":
        ["E<sign>?<digit_sequence>","e<sign>?<digit_sequence>"],  
    "unsigned_digit_sequence":
        ["<digit>+"],
    "<digit_sequence>":
        ["<sign>?<unsigned_digit_sequence>"],
    "<sign>":
        [" +", " -"],
    "<letter>":
        srange(string.ascii_letters),
    "<digit>":
        srange(string.digits),
    "<string>":
        ["'<string_character>+'"],
    "<string_character>":
        srange(CHARACTERS_WITHOUT_QUOTE) + ["''"],  # ?
    "<label>":
        ["integer_number"],
    "<constant>":
        [""],  # TODO: [ sign ] (constant_identifier | number) | string
}

VARIABLE_IDENTIFIER_EBNF: Grammar = {
    "<identifier>": # TODO = letter { letter | digit } 
        ["<letter>+"],
    "<file-variable>":
        ["<variable>"],
    "<referenced-variable>":
        ["<pointer-variable>^"], # ?
    "<record-variable>":
        ["<variable>"],
    "<pointer-variable>":
        ["<variable>"],
    "<actual-variable>":
        ["<variable>"],
    "<array-variable>":
        ["<variable>"],
    "<field-identifier>":
        ["<identifier>"],
    "<constant-identifier>":
        ["<identifier>"],
    "<variable-identifier>":
        ["<identifier>"],
    "<type-identifier>":
        ["<identifier>"],
    "<procedure-identifier>":
        ["<identifier>"],
    "<function-identifier>":
        ["<identifier>"],
    "<bound-identifier>":
        ["<identifier>"],
}

# INPUT_OUTPUT_EBNF 
# Record Fields
# Types

PASCAL_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<variable-list>"],
    **LOW_LEVEL_EBNF,
}
# assert is_valid_grammar(PASCAL_EBNF_GRAMMAR)
pascal_grammar = convert_ebnf_grammar(PASCAL_EBNF_GRAMMAR)
# assert is_valid_grammar(pascal_grammar)


### ST 语言 EBNF 自顶而下构建



1. 先生成变量列表
2. 根据变量列表生成程序主体

*时间有限，此处构建的仅为 ST 语言的子集*

范式：

```iecst
PROGRAM program0
  VAR
    // 生成变量列表
  END_VAR
  // 生成程序主体
END_PROGRAM


CONFIGURATION Config0

  RESOURCE Res0 ON PLC
    TASK task0(INTERVAL := T#20ms,PRIORITY := 0);
    PROGRAM instance0 WITH task0 : program0;
  END_RESOURCE
END_CONFIGURATION
```


#### 变量列表 EBNF

In [108]:
ST_VAR_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<变量声明语句>+"],
    "<变量声明语句>":
        ["<variable-name> : <variable-type>;\n"],
    "<variable-name>": # 形如 Var233
        ["Var<digit>+"],
    "<variable-type>": # 形如 INT REAL
        ["INT", "REAL", "STRING", "BOOL"],
    "<digit>":
        srange(string.digits)
} # 之后用随机选取变量名和类型的方法替代了

#### 程序主体 EBNF

In [109]:
ST_BODY_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<StatementList>"],
    **ST_STATEMENTS_EBNF,
    **ST_EXPRESSIONS_EBNF,
    **ST_ASSIGNMENTS_EBNF,
    **ST_ITERATION_STATEMENT_EBNF,
    **ST_SELECTION_STATEMENT_EBNF,
}
assert is_valid_grammar(ST_BODY_EBNF_GRAMMAR)

## ST 生成

> 注意 `GrammarCoverageFuzzer` 的使用
>
> Subsequent calls to `fuzz()` will go for further coverage (i.e., covering the other area code digits, for example); a call to `reset()` clears the recored coverage, starting anew.

In [110]:
ST_DATA_TYPES = ["INT", "REAL", "BOOL"]
ST_DATA_TYPES_W = [0.1,0.1,0.8]
assert sum(ST_DATA_TYPES_W) == 1

st_vars = {}
for i in range(np.random.randint(1,10)):
    st_vars[f'Var{np.random.randint(100)}'] = np.random.choice(ST_DATA_TYPES, p=ST_DATA_TYPES_W)
st_vars_s = ""
for i in st_vars:
    st_vars_s += f"        {i} : {st_vars[i]};\n"
    
# print(st_vars_s)
# print([*st_vars])

In [111]:
ST_BODY_EBNF_GRAMMAR["<Variable>"] = [*st_vars]
fbody = GrammarCoverageFuzzer(convert_ebnf_grammar(ST_BODY_EBNF_GRAMMAR),start_symbol="<start>",max_nonterminals=200)
body_s = fbody.fuzz()
# print(body_s)

In [112]:
# 块拼接
MODEL_STR = f"""
PROGRAM program0
  VAR
{st_vars_s}
  END_VAR
  
{body_s}

END_PROGRAM
"""

# 写入文件
with open("st_gen.st", "w") as f:
    f.write(MODEL_STR)
print(MODEL_STR)


PROGRAM program0
  VAR
        Var92 : REAL;

  END_VAR
  
Var92 := (9. -54 / 9.3 - Var92 / 2.2 + 3 / 1.7 * Var92) *  -9 - 61. NOT 785 *  -2. -0 * 3.6 / 1.6 / Var92 * Var92 * Var92 * Var92   AND 6.75 * 7 / Var92 / Var92 + Var92 * Var92- Var92 >  NOT 5.7 + Var92- Var92>= Var92 / Var92 >= Var92 < Var92  = 2 * 8 * Var92 / Var92 - Var92+ Var92 < Var92 + Var92- Var92<= Var92 = Var92 / Var92 + Var92 < Var92 = Var92  > Var92 = Var92  = Var92  = Var92    

;

FOR Var92 := Var92   = Var92   AND Var92    XOR Var92    XOR Var92    XOR Var92     OR Var92    AND Var92   AND Var92    XOR Var92    OR Var92     XOR Var92    OR Var92     OR Var92      TO Var92    AND Var92   AND Var92    XOR Var92     OR Var92    AND Var92    XOR Var92    XOR Var92    OR Var92     XOR Var92     BY Var92    AND Var92    XOR Var92    XOR Var92    XOR Var92     OR Var92     OR Var92     OR Var92      DO
    
CASE Var92     XOR Var92     OR Var92     OR Var92      OF
    1  : EXIT;

8  : EXIT;
EXIT;
EXIT;


ELSE
    
IF V

## 测试

### 单个测试

In [113]:
program = "./iec2c"
FILE = "st_gen.st"
arg = [program,
       "-f",
       "-l",
       "-p",
       "-I", "/home/hxn/下载/OpenPLC_Editor/matiec/lib",
       "-T", "/home/hxn/桌面/ST_study/openPLC_project_test/build",
       FILE]
# -f // 在错误消息上显示完整的位置
# -l // 使用宽松的数据类型等价模型       (a non-standard extension?)
# -p // 允许使用前向引用               (a non-standard extension?)
# -I // include_directory
# -T // target_directory
result = subprocess.run(arg,
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7

print(">>>stdout")
print(result.stdout)
print(">>>stderr")
print(result.stderr)
print(">>>returncode")
print(result.returncode)

>>>stdout

>>>stderr
st_gen.st:8-12..8-13: error: ')' missing at the end of expression in ST expression.
st_gen.st:8-12..8-13: error: ';' missing at the end of statement in ST statement.
st_gen.st:8-13..8-388: error: invalid statement in ST statement.

3 error(s) found. Bailing out!

>>>returncode
1


### 循环测试

In [114]:

tempdir = tempfile.mkdtemp()
program = "./iec2c"
arg = [program,
       "-f",
       "-l",
       "-p",
       "-I", "./lib",
       "-T", tempdir]
# -f // 在错误消息上显示完整的位置
# -l // 使用宽松的数据类型等价模型       (a non-standard extension?)
# -p // 允许使用前向引用               (a non-standard extension?)
# -I // include_directory
# -T // target_directory
ST_DATA_TYPES_W = [0,0,1]
REPEAT_TIMES = 20
retuencodes = []
!rm -r ./AFL_testcase/st_program_test*
for i in range(REPEAT_TIMES):
    
    # 生成随机变量
    st_vars = {}
    for j in range(np.random.randint(2,4)):
        st_vars[f'Var{np.random.randint(100)}'] = np.random.choice(ST_DATA_TYPES, p=ST_DATA_TYPES_W)
    st_vars_s = ""
    for j in st_vars:
        st_vars_s += f"        {j} : {st_vars[j]};\n"
    
    ST_BODY_EBNF_GRAMMAR["<Variable>"] = [*st_vars]
    # NOTE 理应在循环外建立对象保证效率和Coverage，但是这样会导致变量名和类型不变
    fbody = GrammarCoverageFuzzer(convert_ebnf_grammar(ST_BODY_EBNF_GRAMMAR),start_symbol="<start>",max_nonterminals=10)

    # 块拼接
    MODEL_STR = f"""
PROGRAM program0
    VAR
{st_vars_s}
    END_VAR

{fbody.fuzz()}

END_PROGRAM
"""
    # 写入文件
    FILE = f'./AFL_testcase/st_program_test{i}.st'
    with open(FILE, "w") as f:
        f.write(MODEL_STR)
    
    result = subprocess.run(arg + [FILE],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)  # Will be "text" in Python 3.7

    # print(result.returncode)
    retuencodes.append(result.returncode)
    if result.returncode not in [0]:
        print(">>>stdout")
        print(result.stdout)
        print(">>>stderr")
        print(result.stderr)
        print(">>>returncode")
        print(result.returncode)
        print(">>>FILE")
        print(MODEL_STR)
    if i % 50 == 0:
        print(f"{i}/{REPEAT_TIMES} done")
print("DONE, pass rate: ",retuencodes.count(0)/REPEAT_TIMES)


0/20 done
>>>stdout

>>>stderr
./AFL_testcase/st_program_test1.st:10-6..10-10: error: Invalid data type for 'FOR' control variable.
./AFL_testcase/st_program_test1.st:10-21..10-25: error: Invalid data type for 'FOR' begin expression.
./AFL_testcase/st_program_test1.st:10-37..10-41: error: Invalid data type for 'FOR' end expression.
./AFL_testcase/st_program_test1.st:15-11..15-24: error: Data type mismatch for '*' expression.
4 error(s) found. Bailing out!

>>>returncode
1
>>>FILE

PROGRAM program0
    VAR
        Var72 : BOOL;
        Var70 : BOOL;

    END_VAR


FOR Var70 := Var72       TO Var72        DO
    EXIT;
EXIT;

END_FOR;
Var72 := Var70 * Var72  <= Var70     

;


END_PROGRAM

>>>stdout

>>>stderr
./AFL_testcase/st_program_test4.st:11-13..11-17: error: 'CASE' quantity not an integer or enumerated.
1 error(s) found. Bailing out!

>>>returncode
1
>>>FILE

PROGRAM program0
    VAR
        Var30 : BOOL;
        Var31 : BOOL;
        Var10 : BOOL;

    END_VAR


CASE Var10       O

根据语法规则生成的输入很难找到 iec2c 翻译器的漏洞，因为语法规则本身就是正确的，很难产生崩溃等异常情况。

找到的漏洞多为语义上的漏洞，而不是程序运行中的崩溃情况


## 进一步的测试

考虑白盒测试

通过 AFL 工具使用交叉变异，源码插桩的方式对 openplc 进行更深入的模糊测试。

源码插桩：

```bash
$ ./configure CC=afl-gcc CXX=afl-g++ 
$ make

```

筛选用例：

```bash
afl-cmin -i ./AFL_testcase -o ./AFL_testcase_cmin ./matiec_src/iec2c @@ -f -l -p
```

执行模糊测试：

```bash
su
sudo echo core >/proc/sys/kernel/core_pattern
cd /sys/devices/system/cpu
echo performance | tee cpu*/cpufreq/scaling_governor

$ afl-fuzz -i ./AFL_testcase -o ./AFL_findings ./matiec_src/iec2c @@ -f -l -p

```
通过并行化的方式加快测试速度：

```bash
afl-fuzz -i ./AFL_testcase -o ./AFL_sync_dir -M fuzzer01 ./matiec_src/iec2c @@ -f -l -p
afl-fuzz -i ./AFL_testcase -o ./AFL_sync_dir -S fuzzer02 ./matiec_src/iec2c @@ -f -l -p
afl-fuzz -i ./AFL_testcase -o ./AFL_sync_dir -S fuzzer03 ./matiec_src/iec2c @@ -f -l -p
afl-fuzz -i ./AFL_testcase -o ./AFL_sync_dir -S fuzzer04 ./matiec_src/iec2c @@ -f -l -p

...

```

亦可中断后继续上一次回话

最小化测试用例：

```bash
afl-tmin -i ./crashes_bin -o ./crashes_bin_tmin ./matiec_src/iec2c @@ -f -l -p
```


虽然电脑配置不太好，但是很快就发现了几个程序崩溃：

![image-20220815233222389](STgenREADME/image-20220815233222389.png)


## 发现的一些漏洞

通过以下语句精简测试样本：

```bash
afl-tmin -i ./crashes_bin/样本  -o ./crashes_bin_tmin/目标.st ./matiec_src/iec2c @@ -f -l -p
```

### 语义类漏洞：

### 段错误漏洞：



```iecst
PROGRAM 
    VAR
        r:BOOL;
        V0000:BOOL;
    END_VAR
EXIT;
V0000:=0;
END_PROGRAM
```



![image-20220815200544739](STgenREADME/image-20220815200544739.png)

想不到这个看似正常的程序会导致崩溃。

```iecst
PROGRAM q
END_PROGRAM
```

```iecst
PROGRAM VAR
V00000:BOOL;r:BOOL;r0:BOOL;END_VAR
V00000:=0;END_PROGRAM
```

```iecst
PROGRAM VAR
r:BOOL;END_VAR
REPEAT
CASE 0OF 0:EXIT;END_CASE;UNTIL END_REPEAT;END_PROGRAM
```

以上面这个程序为例，进行调试：

在源码中加入调试信息

```bash
./configure CFLAGS="-g"
make
```

使用 gbd 进行调试发现异常所在点


![image-20220815212229654](STgenREADME/image-20220815212229654.png)

所在的源码位置

`absyntax/visitor.cc`

```c
void *iterator_visitor_c::visit_list(list_c *list) {
  for(int i = 0; i < list->n; i++) {
    list->elements[i]->accept(*this);
  }
  return NULL;
}
```

![image-20220815233038951](STgenREADME/image-20220815233038951.png)

探查变量发现是下标溢出错误。

时间有限，就不展开了。

其他的漏洞也是在相似的语句发生，但是是在源程序中的不同位置。

### 未响应漏洞

*电脑比较渣...很多未响应的用例经过检查后发现是电脑算力抖动导致的，就先不考虑了*




## 总结与展望

### 总结





### 下一步的工作

- [ ] 通过对 iec2c 的源码分析，找到 bug 的原因
- [ ] 更完善的语法生成，使得生成的样本更加符合实际的 PLC 程序，如类型匹配等
- [ ] 时间不够甚至AFL的一轮循环都没有跑完。完整地进行一遍 AFL 的测试，找到更完整的漏洞
- [ ] 对 openplc 的其他部分进行漏洞挖掘

## 参考资料

[EBNF for Pascal](http://www.cs.kent.edu/~durand/CS43101Fall2004/resources/Pascal-EBNF.html)

[ST 语法参考](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.49.2016&rep=rep1&type=pdf)

https://zh.wikipedia.org/wiki/%E7%B5%90%E6%A7%8B%E5%BC%8F%E6%96%87%E4%BB%B6%E7%B7%A8%E7%A8%8B%E8%AA%9E%E8%A8%80

https://github.com/thiagoralves/OpenPLC_v3

https://en.wikipedia.org/wiki/IEC_61131-3

https://github.com/nucleron/matiec